In [1]:
# This cell is used to setup the Python environment.  It only needs to run once.

# Import the slip report code from slip_report_funcs.py in this directory 
import slip_report_funcs as sr
sr.Setup()

2026-02-24 12:51:01,890 (INFO) NumExpr defaulting to 10 threads.


Loading BokehJS ...

2026-02-24 12:51:02,189 (INFO) Plotting function loaded.
2026-02-24 12:51:02,189 (INFO) Slip report initialized.


In [10]:
# This cell sets up the report criteria.  When you alter any of these values, re-run the cell.

# Telescope, specify either 1 or 2
telescope = 1

# Telescope azimuth rotation.  The default graph plots AZ of 0 pointing to the right.  
# Use this to rotate it to match the typical diagrams of the building.
azimuth_rotate = -45

# Time range.  Choices are:
#   days_ago = 15    (Look backwards from today, this number of days.)
#       OR
#   days_ago = 0     (Set this to zero to use a specific start and end date.)
#   start_date = '20251217'
#   end_date = '20260122'

days_ago = 0
# OR...
start_date = '20251201'
end_date = '20251210'


In [11]:
# This cell computes the the time range of your query and plots it below.

if days_ago > 0:

    today = sr.datetime.date.today()
    dt_today = sr.datetime.datetime(today.year, today.month, today.day, 0, 0)

    t0 = dt_today - sr.datetime.timedelta(days=days_ago)
    t1 = dt_today + sr.datetime.timedelta(days=1)

else:
    import dateutil.parser
    t0 = dateutil.parser.parse(start_date)
    t1 = dateutil.parser.parse(end_date) 

# Get the data for the range we are looking at
all_slips = sr.GetAllSlips(telescope, t0, t1)
reduced = sr.ReduceSlipData(all_slips)
sr.PlotSlips(reduced, top=azimuth_rotate)


2026-02-24 13:45:00,578 (INFO) Motor slip analysis for 1: 2025-12-01 00:00:00 to 2025-12-10 00:00:00
2026-02-24 13:45:02,760 (WARNING) No motor slips found for k1:dcs:axe:az:mtr1ASlip at 2025-12-01T00:00:00-10:00
2026-02-24 13:45:04,719 (WARNING) No motor slips found for k1:dcs:axe:az:mtr1BSlip at 2025-12-01T00:00:00-10:00
2026-02-24 13:45:06,883 (WARNING) No motor slips found for k1:dcs:axe:az:mtr2ASlip at 2025-12-01T00:00:00-10:00
2026-02-24 13:45:08,760 (WARNING) No motor slips found for k1:dcs:axe:az:mtr2BSlip at 2025-12-01T00:00:00-10:00
2026-02-24 13:45:10,707 (WARNING) No motor slips found for k1:dcs:axe:az:mtr3ASlip at 2025-12-01T00:00:00-10:00
2026-02-24 13:45:12,742 (DEBUG) Starting new HTTP connection (1): k1epicsgateway:17668
2026-02-24 13:45:15,656 (DEBUG) http://k1epicsgateway:17668 "POST /retrieval/data/getDataAtTime.json?at=2025-12-06T21%3A43%3A34.065712929-00%3A00 HTTP/1.1" 200 259
2026-02-24 13:45:15,660 (DEBUG) Starting new HTTP connection (1): k1epicsgateway:17668
2

RuntimeError: Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.

In [35]:
# Run this cell to show the slips in table format
import datetime

# Iterate each row of the data frame and build up the counts
print(f'UT Date, UT Time, HST date, HST time, Tel Az, Leg 1 Az, Leg 2 Az, Leg 3 Az, Leg 4 Az, CSS Device')
for index, row in all_slips.iterrows():
    motor = row['motor']
    pos = row['pos']
    sampletime = row['sampletime']

    # Localize the times to UTC and HST
    utc = sampletime.tz_localize('UTC')
    hst = utc.tz_convert('HST')
    utc_date = utc.strftime('%Y-%m-%d')
    utc_time = utc.strftime('%H:%M:%S')
    hst_date = hst.strftime('%Y-%m-%d')
    hst_time = hst.strftime('%H:%M:%S')

    # Compute the leg locations
    leg1 = (pos + 45) % 360
    leg2 = (leg1 + 90) % 360
    leg3 = (leg2 + 90) % 360
    leg4 = (leg3 + 90) % 360
    
    print(f'{utc_date},{utc_time},{hst_date},{hst_time},{pos},{leg1},{leg2},{leg3},{leg4},{motor}')


UT Date, UT Time, HST date, HST time, Tel Az, Leg 1 Az, Leg 2 Az, Leg 3 Az, Leg 4 Az, CSS Device
2025-12-06,21:43:34,2025-12-06,11:43:34,150,195,285,15,105,3B
2025-12-08,00:18:38,2025-12-07,14:18:38,1,46,136,226,316,3B


In [ ]:
# Run this cell to re-plot the retrieved data
sr.PlotSlips(reduced, flip=True, top=-45)

In [ ]:
# Run this cell to plot demo data, no retrieval needed
sr.PlotSlips(sr.demo_data, flip=True, top=azimuth_rotate)